In [1]:
import os
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph

load_dotenv()

os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")

graph=Neo4jGraph(url=os.getenv("NEO4J_URI"),username=os.getenv("NEO4J_USERNAME"),password=os.getenv("NEO4J_PASSWORD"),refresh_schema=False )

C:\Users\SOMYA\AppData\Local\Temp\ipykernel_23460\2530791801.py:11: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the `langchain-neo4j package and should be used instead. To use it run `pip install -U `langchain-neo4j` and import as `from `langchain_neo4j import Neo4jGraph``.
  graph=Neo4jGraph(url=os.getenv("NEO4J_URI"),username=os.getenv("NEO4J_USERNAME"),password=os.getenv("NEO4J_PASSWORD"),refresh_schema=False )


In [12]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)

In [13]:
## Dataset Moview 
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))


"""

In [ ]:
## Alternative to graph.query(moview_query)
with driver.session() as session:
    result = session.run(moview_query)  # moview_query is your Cypher string
    for record in result:
        print(record)

In [2]:
from langchain_groq import ChatGroq

groq_api_key=os.getenv("GROQ_API_KEY")
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    groq_api_key=groq_api_key
)

d:\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True,allow_dangerous_requests=True )
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x00000187AC7B2060>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000187D629BE90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000187D6358800>, model_name=

In [11]:
response=chain.invoke({"query":"What are Movie title?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie) RETURN m.title
Full Context:
[{'m.title': 'Toy Story'}, {'m.title': 'Jumanji'}, {'m.title': 'Grumpier Old Men'}, {'m.title': 'Waiting to Exhale'}, {'m.title': 'Father of the Bride Part II'}, {'m.title': 'Heat'}, {'m.title': 'Sabrina'}, {'m.title': 'Tom and Huck'}, {'m.title': 'Sudden Death'}, {'m.title': 'GoldenEye'}]

> Finished chain.


{'query': 'What are Movie title?',
 'result': 'Toy Story, Jumanji, Grumpier Old Men, Waiting to Exhale, Father of the Bride Part II, Heat, Sabrina, Tom and Huck, Sudden Death, GoldenEye.'}

In [14]:
response=chain.invoke({"query":"Find all Person name?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person) RETURN p.name
Full Context:
[{'p.name': 'John Lasseter'}, {'p.name': 'Joe Johnston'}, {'p.name': 'Howard Deutch'}, {'p.name': 'Forest Whitaker'}, {'p.name': 'Charles Shyer'}, {'p.name': 'Michael Mann'}, {'p.name': 'Sydney Pollack'}, {'p.name': 'Peter Hewitt'}, {'p.name': 'Peter Hyams'}, {'p.name': 'Martin Campbell'}]

> Finished chain.


{'query': 'Find all Person name?',
 'result': 'John Lasseter, Joe Johnston, Howard Deutch, Forest Whitaker, Charles Shyer, Michael Mann, Sydney Pollack, Peter Hewitt, Peter Hyams, Martin Campbell.'}

In [26]:
response=chain.invoke({"query":"Find Person who acted but not directed  Movie Jumanji?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(d:Person) WHERE m.title = "Jumanji" AND p <> d RETURN p
Full Context:
[{'p': {'name': 'Robin Williams'}}, {'p': {'name': 'Bradley Pierce'}}, {'p': {'name': 'Kirsten Dunst'}}, {'p': {'name': 'Jonathan Hyde'}}]

> Finished chain.


{'query': 'Find Person who acted but not directed  Movie Jumanji?',
 'result': 'Jonathan Hyde, Kirsten Dunst, Bradley Pierce acted in Movie Jumanji but not directed it.'}

In [154]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {{name: 'Tom Hanks'}})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Jumanji",
        "query": "MATCH (m:Movie {{title: 'Jumanji'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

In [156]:
# from langchain_core.prompts import FewShotPromptTemplate,PromptTemplate

example_prompt=PromptTemplate.from_template(
    "User input:{question}\n Cypher query:{query}"
)

prompt = FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given an input question, create a syntactically very accurate Cypher query.",
    suffix="User input: {question}\nCypher query: ",
    input_variables=["question"]
)

In [182]:
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    llm=llm,
    cypher_prompt=prompt,
    verbose=True,
    allow_dangerous_requests=True
)

In [183]:
response = chain.run("Which actors played in the movie Casino?")
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a:Person) RETURN a.name

Full Context:
[{'a.name': 'Robert De Niro'}, {'a.name': 'Joe Pesci'}, {'a.name': 'Sharon Stone'}, {'a.name': 'James Woods'}]

> Finished chain.
Robert De Niro, Joe Pesci, Sharon Stone, James Woods played in the movie Casino.


In [177]:
response = chain.invoke("Which actors played in the movie Casino?")
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a:Person) RETURN a.name

Full Context:
[{'a.name': 'Robert De Niro'}, {'a.name': 'Joe Pesci'}, {'a.name': 'Sharon Stone'}, {'a.name': 'James Woods'}]

> Finished chain.
{'query': 'Which actors played in the movie Casino?', 'result': 'Robert De Niro, Joe Pesci, Sharon Stone played in the movie Casino.'}


In [161]:
chain.invoke("actors who acted in multiple movies")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
RETURN a.name, COUNT(m) as movie_count
ORDER BY movie_count DESC

Full Context:
[{'a.name': 'Gene Hackman', 'movie_count': 4}, {'a.name': 'John Travolta', 'movie_count': 4}, {'a.name': 'Julianne Moore', 'movie_count': 4}, {'a.name': 'Robert Downey Jr.', 'movie_count': 4}, {'a.name': 'Christian Slater', 'movie_count': 4}, {'a.name': 'Samuel L. Jackson', 'movie_count': 4}, {'a.name': 'Gary Oldman', 'movie_count': 4}, {'a.name': 'Angela Bassett', 'movie_count': 3}, {'a.name': 'Robert De Niro', 'movie_count': 3}, {'a.name': 'Julia Ormond', 'movie_count': 3}]

> Finished chain.


{'query': 'actors who acted in multiple movies',
 'result': 'Gene Hackman, John Travolta, Julianne Moore, Robert Downey Jr., Christian Slater, Samuel L. Jackson, and Gary Oldman have acted in multiple movies.'}

In [171]:
chain.invoke("List all the genres of the movie Jumanji")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie {title: 'Jumanji'})-[:IN_GENRE]->(g:Genre)
RETURN g.name

Full Context:
[{'g.name': 'Adventure'}, {'g.name': 'Children'}, {'g.name': 'Fantasy'}]

> Finished chain.


{'query': 'List all the genres of the movie Jumanji',
 'result': 'Adventure, Children, Fantasy are genres of the movie Jumanji.'}

In [184]:
print(graph.schema)